## D212 Data Mining 2 PA 3
##### Submitted By Edwin Perry
### Table of Contents
<ol>
    <li><a href="#A">Research Question</a></li>
    <li><a href="#B">Technique Justification</a></li>
    <li><a href="#C">Data Preparation</a></li>
    <li><a href="#D">Analysis</a></li>
    <li><a href="#E">Data Summary and Implications</a></li>
    <li><a href="#F">Sources</a></li>
</ol>
<h4 id="A">Research Question</h4>
    <h5>Providing Question</h5>
    <p>The research question that I have decided to use for this project is "What product offerings are positively associated with the Dust-Off Compressed Gas 2 pack?" This can be used to items commonly purchased together to offer deals, discounts, and bundles.</p>
    <h5>Objectives and Goals</h5>
    <p>Determining common groupings of goods provided can assist the telecommunications business in creating bundles and discount groups, encouraging customers to purchase these goods from the business.If customers can purchase these bundles from our business, we can both improve customer satisfaction with the business, encouraging return customers, and ensure customers purchase our goods, rather than purhcasing these goods from competitors. I am specifically choosing to analyze the gas 2 pack because it is a lighter item that would be expected to be purchased alongside other, larger purchases.</p>
    <h4 id="B">Technique Justification</h4>
    <h5>Market Basket Description</h5>
    <p>Market Basket Analysis is a way to identify the most common groupings of products purchased. This is often used to create recommendations based on these groupings, allowing a business to create bundles or deals on the basis of these common groupings. The analysis is done by creating rules with an antecedent and consequent relationship. To put it in simple terms, we determine a value (the antecedent) and see if it has a causal relationship with the purchase of another item (the consequent).  To bring it back to the research question, I would anticipate that the result of this analysis will reveal a relationship that could be formulated as "If a customer purchases the router, then they are likely to purchase X" or "customers that purchase X are more likely to purchase a router"</p>
    <h5>Example Transaction</h5>

In [153]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import association_rules, apriori
from mlxtend.preprocessing import TransactionEncoder

df = pd.read_csv('./Data Source/teleco_market_basket.csv')
df.iloc[3]

Item01    Apple Lightning to Digital AV Adapter
Item02         TP-Link AC1750 Smart WiFi Router
Item03                             Apple Pencil
Item04                                      NaN
Item05                                      NaN
Item06                                      NaN
Item07                                      NaN
Item08                                      NaN
Item09                                      NaN
Item10                                      NaN
Item11                                      NaN
Item12                                      NaN
Item13                                      NaN
Item14                                      NaN
Item15                                      NaN
Item16                                      NaN
Item17                                      NaN
Item18                                      NaN
Item19                                      NaN
Item20                                      NaN
Name: 3, dtype: object

<p>We can see that the customer can have up to 20 items in one transaction. This row shows the customer purchasing 3 items: an Apple Lightning Adapter, a TP-Link WiFi Router, and an Apple Pencil. Every row within this dataset contains similar data for customer purchases.</p>
<h5>Assumption</h5>
<p>The usage of market basket analysis requires certain assumptions to be made. One such assumption is that there is a complimentary or causal relationship between the items a customer purchases. As mentioned before, this analysis assumes that we can formulate the relationship between certain items as "A customer buying X will likely purchase Y," which means that there needs to be some factor causing these 2 products to be purchased alongside each other.</p>
<h4 id="C">Data Preparation and Analysis</h4>
<h5>Data Preparation</h5>
<p>The data is already loaded into the dataframe (as this was one step to display the row above). Next, we need to clean the data such that we can perform our analysis. First, we will print the head of the dataframe, to allow us to get a view of the data.</p>

In [154]:
pd.set_option("display.max_columns", None)
df.head(10)

,Item01,Item02,Item03,Item04,Item05,Item06,Item07,Item08,Item09,Item10,Item11,Item12,Item13,Item14,Item15,Item16,Item17,Item18,Item19,Item20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Logitech M510 Wireless mouse,HP 63 Ink,HP 65 ink,nonda USB C to USB Adapter,10ft iPHone Charger Cable,HP 902XL ink,Creative Pebble 2.0 Speakers,Cleaning Gel Universal Dust Cleaner,Micro Center 32GB Memory card,YUNSONG 3pack 6ft Nylon Lightning Cable,TopMate C5 Laptop Cooler pad,Apple USB-C Charger cable,HyperX Cloud Stinger Headset,TONOR USB Gaming Microphone,Dust-Off Compressed Gas 2 pack,3A USB Type C Cable 3 pack 6FT,HOVAMP iPhone charger,SanDisk Ultra 128GB card,FEEL2NICE 5 pack 10ft Lighning cable,FEIYOLD Blue light Blocking Glasses
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple Lightning to Digital AV Adapter,TP-Link AC1750 Smart WiFi Router,Apple Pencil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,UNEN Mfi Certified 5-pack Lightning Cable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Cat8 Ethernet Cable,HP 65 ink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Dust-Off Compressed Gas 2 pack,Screen Mom Screen Cleaner kit,Moread HDMI to VGA Adapter,HP 62XL Tri-Color ink,Apple USB-C Charger cable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<p>We can see that, for some reason, the data is stored in a manner that it contains a line of entirely null values in between every entry. As a result, the first thing we will do is remove these blank rows, by filtering the Item01 column such that the NaN values are filtered out.</p>

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002 entries, 0 to 15001
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Item01  7501 non-null   object
 1   Item02  5747 non-null   object
 2   Item03  4389 non-null   object
 3   Item04  3345 non-null   object
 4   Item05  2529 non-null   object
 5   Item06  1864 non-null   object
 6   Item07  1369 non-null   object
 7   Item08  981 non-null    object
 8   Item09  654 non-null    object
 9   Item10  395 non-null    object
 10  Item11  256 non-null    object
 11  Item12  154 non-null    object
 12  Item13  87 non-null     object
 13  Item14  47 non-null     object
 14  Item15  25 non-null     object
 15  Item16  8 non-null      object
 16  Item17  4 non-null      object
 17  Item18  4 non-null      object
 18  Item19  3 non-null      object
 19  Item20  1 non-null      object
dtypes: object(20)
memory usage: 2.3+ MB


In [156]:
df = df[df['Item01'].notna()]
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7501 entries, 0 to 7500
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Item01  7501 non-null   object
 1   Item02  5747 non-null   object
 2   Item03  4389 non-null   object
 3   Item04  3345 non-null   object
 4   Item05  2529 non-null   object
 5   Item06  1864 non-null   object
 6   Item07  1369 non-null   object
 7   Item08  981 non-null    object
 8   Item09  654 non-null    object
 9   Item10  395 non-null    object
 10  Item11  256 non-null    object
 11  Item12  154 non-null    object
 12  Item13  87 non-null     object
 13  Item14  47 non-null     object
 14  Item15  25 non-null     object
 15  Item16  8 non-null      object
 16  Item17  4 non-null      object
 17  Item18  4 non-null      object
 18  Item19  3 non-null      object
 19  Item20  1 non-null      object
dtypes: object(20)
memory usage: 1.1+ MB


<p>Now that the data has been filtered down, we need to convert the dataframe into a list of lists, where each list represents a transaction. Finally, we use a TransactionEncoder to perform one-hot encoding on this list to create a dataframe.</p>

In [157]:
transactions = []
for i in range(0, df.shape[0]):
    transaction = df.iloc[i].dropna().tolist()
    transactions.append(transaction)

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

df_encoded

,10ft iPHone Charger Cable,10ft iPHone Charger Cable 2 Pack,3 pack Nylon Braided Lightning Cable,3A USB Type C Cable 3 pack 6FT,5pack Nylon Braided USB C cables,ARRIS SURFboard SB8200 Cable Modem,Anker 2-in-1 USB Card Reader,Anker 4-port USB hub,Anker USB C to HDMI Adapter,Apple Lightning to Digital AV Adapter,Apple Lightning to USB cable,Apple Magic Mouse 2,Apple Pencil,Apple Pencil 2nd Gen,Apple Power Adapter Extension Cable,Apple USB-C Charger cable,AutoFocus 1080p Webcam,BENGOO G90000 headset,Blue Light Blocking Glasses,Blue Light Blocking Glasses 2pack,Brother Genuine High Yield Toner Cartridge,Cat 6 Ethernet Cable 50ft,Cat8 Ethernet Cable,CicTsing MM057 2.4G Wireless Mouse,Cleaning Gel Universal Dust Cleaner,Creative Pebble 2.0 Speakers,DisplayPort ot HDMI adapter,Dust-Off Compressed Gas,Dust-Off Compressed Gas 2 pack,FEEL2NICE 5 pack 10ft Lighning cable,FEIYOLD Blue light Blocking Glasses,Falcon Dust Off Compressed Gas,HOVAMP Mfi 6pack Lightning Cable,HOVAMP iPhone charger,HP 61 2 pack ink,HP 61 Tri-color ink,HP 61 ink,HP 62XL Tri-Color ink,HP 62XL ink,HP 63 Ink,HP 63 Tri-color ink,HP 63XL Ink,HP 63XL Tri-color ink,HP 64 Tri-Color ink,HP 64 ink,HP 65 ink,HP 902XL ink,HP 952 ink,HP ENVY 5055 printer,HP952XL ink,HooToo USB C Hub,HyperX Cloud Stinger Headset,Jelly Comb 2.4G Slim Wireless mouse,Leader Desk Pad Protector,Logitech M510 Wireless mouse,Logitech MK270 Wireless Keyboard/Mouse,Logitech MK345 Wireless combo,Logitech USB H390 headset,M.2 Screw kit,Mfi-Certified Lightning to USB A Cable,Micro Center 32GB Memory card,Microsot Surface Dock 2,Moread HDMI to VGA Adapter,Mpow HC6 USB Headset,NETGEAR CM500 Cable Modem,NETGEAR Nighthawk WiFi Router,NETGEAR Orbi Home Mesh WiFi System,Nylon Braided Lightning to USB cable,PS4 Headset,Premium Nylon USB Cable,RUNMUS Gaming Headset,SAMSUNG 128GB card,SAMSUNG 256 GB card,SAMSUNG EVO 32GB card,SAMSUNG EVO 64GB card,Sabrent 4-port USB 3.0 hub,SanDisk 128GB Ultra microSDXC card,SanDisk 128GB card,SanDisk 128GB microSDXC card,SanDisk 32GB Ultra SDHC card,SanDisk 32GB card,SanDisk Extreme 128GB card,SanDisk Extreme 256GB card,SanDisk Extreme 32GB 2pack card,SanDisk Extreme Pro 128GB card,SanDisk Extreme Pro 64GB card,SanDisk Ultra 128GB card,SanDisk Ultra 256GB card,SanDisk Ultra 400GB card,SanDisk Ultra 64GB card,Screen Mom Screen Cleaner kit,Stylus Pen for iPad,Syntech USB C to USB Adapter,TONOR USB Gaming Microphone,TP-Link AC1750 Smart WiFi Router,TP-Link AC4000 WiFi router,TopMate C5 Laptop Cooler pad,UNEN Mfi Certified 5-pack Lightning Cable,USB 2.0 Printer cable,USB C to USB Male Adapter,USB Type C Cable,USB Type C to USB-A Charger cable,VIVO Dual LCD Monitor Desk mount,VicTsing Mouse Pad,VicTsing Wireless mouse,Vsco 70 pack stickers,Webcam with Microphone,XPOWER A-2 Air Pump blower,YUNSONG 3pack 6ft Nylon Lightning Cable,hP 65 Tri-color ink,iFixit Pro Tech Toolkit,iPhone 11 case,iPhone 12 Charger cable,iPhone 12 Pro case,iPhone 12 case,iPhone Charger Cable Anker 6ft,iPhone SE case,nonda USB C to USB Adapter,seenda Wireless mouse
0,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,True,False,False,True,True,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,True,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F

<p>The data is now cleaned and prepared, and the below code generates an export of the cleaned data. This data will be provided in this submission</p>

In [158]:
df_encoded.to_csv('./D212P3CleanedData.csv')

<h5>Apriori Algorithm and Association Rules</h5>
<p>To generate the association rules, there are 2 steps that must be followed. First, we use apriori to pull any association that meets a user-provided threshold, which we will determing to be 0.05. Then, we will use association_rules to flesh out the rules and reducing the itemsets further, with a minimum list threshold of 1</p> 

In [159]:
frequent_pairs = apriori(df_encoded, min_support=0.02, use_colnames=True)
rules = association_rules(frequent_pairs, metric='lift', min_threshold=1.0)
rules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   antecedents         94 non-null     object 
 1   consequents         94 non-null     object 
 2   antecedent support  94 non-null     float64
 3   consequent support  94 non-null     float64
 4   support             94 non-null     float64
 5   confidence          94 non-null     float64
 6   lift                94 non-null     float64
 7   representativity    94 non-null     float64
 8   leverage            94 non-null     float64
 9   conviction          94 non-null     float64
 10  zhangs_metric       94 non-null     float64
 11  jaccard             94 non-null     float64
 12  certainty           94 non-null     float64
 13  kulczynski          94 non-null     float64
dtypes: float64(12), object(2)
memory usage: 10.4+ KB


<p>With this, the dataset is appropriately filtered. I chose these minimum thresholds to ensure that we sufficiently restrict the dataset to show very strong associations. As a result, from this dataset of over 7500 transactions, we find a total of 94 relationships.</p>
<h5>Rules Table</h5>
<p>The next step is to provide the association rules table. This table will provide scores for support, confidence, and lift.</p>

In [160]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Dust-Off Compressed Gas 2 pack),(10ft iPHone Charger Cable 2 Pack),0.238368,0.050527,0.023064,0.096756,1.914955,1.0,0.011020,1.051182,0.627330,0.086760,0.048690,0.276610
1,(10ft iPHone Charger Cable 2 Pack),(Dust-Off Compressed Gas 2 pack),0.050527,0.238368,0.023064,0.456464,1.914955,1.0,0.011020,1.401255,0.503221,0.086760,0.286354,0.276610
2,(Dust-Off Compressed Gas 2 pack),(Anker USB C to HDMI Adapter),0.238368,0.068391,0.024397,0.102349,1.496530,1.0,0.008095,1.037830,0.435627,0.086402,0.036451,0.229537
3,(Anker USB C to HDMI Adapter),(Dust-Off Compressed Gas 2 pack),0.068391,0.238368,0.024397,0.356725,1.496530,1.0,0.008095,1.183991,0.356144,0.086402,0.155399,0.229537
4,(Anker USB C to HDMI Adapter),(VIVO Dual LCD Monitor Desk mount),0.068391,0.174110,0.020931,0.306043,1.757755,1.0,0.009023,1.190117,0.462740,0.094465,0.159746,0.213129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,(VIVO Dual LCD Monitor Desk mount),(Screen Mom Screen Cleaner kit),0.174110,0.129583,0.035462,0.203675,1.571779,1.0,0.012900,1.093043,0.440468,0.132207,0.085123,0.238669
90,(Stylus Pen for iPad),(USB 2.0 Printer cable),0.095054,0.170911,0.020131,0.211781,1.239135,1.0,0.003885,1.051852,0.213256,0.081887,0.049296,0.164783
91,(USB 2.0 Printer cable),(Stylus Pen for iPad),0.170911,0.095054,0.020131,0.117785,1.239135,1.0,0.003885,1.025766,0.232768,0.081887,0.025118,0.164783
92,(VIVO Dual LCD Monitor Desk mount),(Stylus Pen for iPad),0.174110,0.095054,0.025197,0.144717,1.522468,1.0,0.008647,1.058066,0.415518,0.103279,0.054879,0.204897


<h5>Top Three Rules</h5>
<p>The rubric also requires the top 3 rules to be explicitly provided, and they can be seen below. We are specifically using lift and confidence to determine this, ranking the rules on the basis of these 2 metrics.</p>

In [161]:
# Sort rules by Lift (descending), then Confidence (descending)
top_rules = rules.sort_values(by=['lift', 'confidence'], ascending=[False, False]).head(3)

# Display top 3 relevant rules
top_rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
84,(VIVO Dual LCD Monitor Desk mount),(SanDisk Ultra 64GB card),0.174110,0.098254,0.039195,0.225115,2.291162,1.0,0.022088,1.163716,0.682343,0.168096,0.140684,0.312015
85,(SanDisk Ultra 64GB card),(VIVO Dual LCD Monitor Desk mount),0.098254,0.174110,0.039195,0.398915,2.291162,1.0,0.022088,1.373997,0.624943,0.168096,0.272197,0.312015
64,(FEIYOLD Blue light Blocking Glasses),(VIVO Dual LCD Monitor Desk mount),0.065858,0.174110,0.022930,0.348178,1.999758,1.0,0.011464,1.267048,0.535186,0.105651,0.210764,0.239939


<h4 id="D">Summary and Implications</h4>
<h5> Metric Significance</h5>
<p>With the rules created, we can now return to our attempts to answer the initial question, regarding products positively associated with the purchase of compressed gas 2 pack. As such, we will filter down the dataframe to only those rules that are relevant for the research question.</p>

In [162]:
antecedent_df = rules[rules['antecedents'] == {'Dust-Off Compressed Gas 2 pack'}]
consequent_df = rules[rules['consequents'] == {'Dust-Off Compressed Gas 2 pack'}]
gas_df = pd.concat([antecedent_df, consequent_df])
gas_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 0 to 63
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   antecedents         36 non-null     object 
 1   consequents         36 non-null     object 
 2   antecedent support  36 non-null     float64
 3   consequent support  36 non-null     float64
 4   support             36 non-null     float64
 5   confidence          36 non-null     float64
 6   lift                36 non-null     float64
 7   representativity    36 non-null     float64
 8   leverage            36 non-null     float64
 9   conviction          36 non-null     float64
 10  zhangs_metric       36 non-null     float64
 11  jaccard             36 non-null     float64
 12  certainty           36 non-null     float64
 13  kulczynski          36 non-null     float64
dtypes: float64(12), object(2)
memory usage: 4.2+ KB


<p>We can see that overall, 36 rules have been created, indicating that a sizable amount of items are sold alongside the compressed gas item we are interested in</p>

In [163]:
print(gas_df['antecedents'].value_counts())
gas_df.sort_values(by=['lift', 'confidence'], ascending=[False, False]).head()

antecedents
(Dust-Off Compressed Gas 2 pack)           18
(Nylon Braided Lightning to USB cable)      1
(TopMate C5 Laptop Cooler pad)              1
(Syntech USB C to USB Adapter)              1
(Stylus Pen for iPad)                       1
(Screen Mom Screen Cleaner kit)             1
(SanDisk Ultra 64GB card)                   1
(SanDisk Ultra 128GB card)                  1
(Premium Nylon USB Cable)                   1
(Logitech M510 Wireless mouse)              1
(10ft iPHone Charger Cable 2 Pack)          1
(HP 62XL Tri-Color ink)                     1
(HP 61 ink)                                 1
(Falcon Dust Off Compressed Gas)            1
(FEIYOLD Blue light Blocking Glasses)       1
(Apple Pencil)                              1
(Apple Lightning to Digital AV Adapter)     1
(Anker USB C to HDMI Adapter)               1
(VIVO Dual LCD Monitor Desk mount)          1
Name: count, dtype: int64


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
1,(10ft iPHone Charger Cable 2 Pack),(Dust-Off Compressed Gas 2 pack),0.050527,0.238368,0.023064,0.456464,1.914955,1.0,0.011020,1.401255,0.503221,0.086760,0.286354,0.276610
0,(Dust-Off Compressed Gas 2 pack),(10ft iPHone Charger Cable 2 Pack),0.238368,0.050527,0.023064,0.096756,1.914955,1.0,0.011020,1.051182,0.627330,0.086760,0.048690,0.276610
37,(FEIYOLD Blue light Blocking Glasses),(Dust-Off Compressed Gas 2 pack),0.065858,0.238368,0.027596,0.419028,1.757904,1.0,0.011898,1.310962,0.461536,0.099759,0.237201,0.267400
36,(Dust-Off Compressed Gas 2 pack),(FEIYOLD Blue light Blocking Glasses),0.238368,0.065858,0.027596,0.115772,1.757904,1.0,0.011898,1.056449,0.566075,0.099759,0.053433,0.267400
52,(Dust-Off Compressed Gas 2 pack),(SanDisk Ultra 64GB card),0.238368,0.098254,0.040928,0.171700,1.747522,1.0,0.017507,1.088672,0.561638,0.138413,0.081449,0.294127


<p>We can summarize the results using 3 key metrics, which are support, confidence, and lift. These scores are contained within the dataframe we've created, but to describe the use of each of these metrics:
<ul>
<li>Support: the proportion of transactions containing the specified rule. For example, 2.3% of transactions contain the 10ft iPhone Charger 2 pack and the compressed gas 2 pack together, as it has a support score of 0.023</li>
<li>Confidence: the proportion of transactions containing the rule, divided by the proportion of transactions containing only the antecedent. For example, the confidence of the previously mentioned transaction has 0.45 confidence with the iPhone chargers as the antecedent, and 0.096 with the compressed gas as the antecedent, indicating that it is more common for the gas to be sold with other items than for the iPhone charger.</li>
<li>Lift: the extent to which the observed relationships exceed the predicted pattern if there were no causal relationship, with 1 indicating no relationship. Anything above 1 is surpassing the expected pattern, indicating a relationship. Looking again at the relationsip between the iPhone charger 2 pack and the gas 2 pack, we see a very high lift, indicating a very strong relationship between the 2.</li>
</ul></p>
<h5>Practical Significance</h5>
<p>We notice a large number of relationships between the compressed gas 2 pack and other products sold, with some of these associations having a very high lift, indicating strong associations between these items. As such, the research question is answered, with products such as iPhone chargers, blue-light filtering glasses, and SD cards having a strong, positive correlation with the compressed gas</p>
<h5>Recommended Action</h5>
<p>The relationships clearly demonstrate that the compressed gas is an item that people are naturally bundling with other items. As such, it would be recommendable to create purchase bundles containing the gas 2-pack and other items in the associated list, such as blue-light filtering glasses and SD cards. Focusing promotional material on the ability to get discounts on compressed gas provided the purchase of any of the associated items would be justifiable, given the strength of these relationships</p>
<h4 id='E'>Multimedia</h4>
<p>A recorded presentation regarding this analysis can be found<a href="https://wgu.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=cfa082b6-2be8-4895-93af-b284001bda44"> here, hosted on Panopto</a></p>
<h4 id='F'>Sources</h4>
<h5>Code References</h5>
<p><a href="https://app.datacamp.com/learn/custom-tracks/custom-data-mining-ii">DataCamp Tutorial</a> used to create the code for the market basket analysis</p>
<h5>Source References</h5>
<p><a href="https://www.geeksforgeeks.org/market-basket-analysis-in-data-mining/">Geeks for Geeks</a> article used to understand the use cases of market basket analysis